In [63]:
import pandas as pd
from vali_tools.valitools import check_health
import os
import numpy as np
import pickle
import datetime
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import mean_squared_error

In [10]:
filename = os.listdir(r'.\feature')[-1]
df = pd.read_parquet(r'.\feature\{}'.format(filename))
df = df[df.survival_rate.notnull()]

In [14]:
check_health(df)

,col_name,null_count,unique_count,row_count,dupe_count,null_pct,unique_pct,dupe_pct,mean,std,min,max,25_pct,50_pct,75_pct,col_types,mode,freq
0,cycle_id,0,2359,2359,0,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,20849,1
1,species_id,0,2,2359,2357,0.000000,0.000848,0.999152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,1.0,2226
2,total_seed,0,824,2359,1535,0.000000,0.349301,0.650699,2.160757e+05,1.800000e+06,1.706124e+05,3.000000e+05,10.000000,90000.000000,1.834640e+05,numeric,NaN,NaN
3,area,0,616,2359,1743,0.000000,0.261128,0.738872,2.270323e+03,4.225000e+05,1.272888e+04,2.303920e+03,1.020000,803.840000,1.380000e+03,numeric,NaN,NaN
4,initial_age,0,36,2359,2323,0.000000,0.015261,0.984739,6.324714e-01,7.500000e+01,4.704696e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,numeric,NaN,NaN
5,subscription_type,0,2,2359,2357,0.000000,0.000848,0.999152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,Free,2194
6,total_seed_type,0,3,2359,2356,0.000000,0.001272,0.998728,NaN,NaN,NaN,NaN,NaN,NaN,NaN,object,net,1182
7,pond_length,0,333,2359,2026,0.000000,0.141162,0.858838,4.357334e+01,6.500000e+02,2.657058e+01,5.183000e+01,3.000000,30.000000,4.013000e+01,numeric,NaN,NaN
8,pond_width,0,376,2359,1983,0.000000,0.159390,0.840610,3.554569e+01,6.500000e+02,2.319543e+01,4.330000e+01,1.500000,26.000000,3.400000e+01,numeric,NaN,NaN
9,pond_depth,0,37,2359,2322,0.000000,0.015685,0.984315,4.367393e+00,3.000000e+02,2.020935e+01,1.500000e+00,0.600000,1.000000,1.250000e+00,numeric,NaN,NaN


# Target 1: Survival Rate

In [86]:
X

,species_id,total_seed,area,initial_age,subscription_type,total_seed_type,pond_length,pond_width,pond_depth,cyc_dur,fasting,cons_fast_count,quantity,cons_feed_count,morning_temperature,morning_salinity,morning_pH,avg_growth_rate
0,1.0,331920,4030.0,0.0,Free,net,65.00,62.00,1.0,107,22.0,8.0,4660.0,107.0,27.848000,24.219512,7.885714,1.77
1,1.0,40000,399.0,0.0,Free,net,30.00,20.00,1.0,58,1.0,1.0,361.5,58.0,27.941579,21.892308,8.592895,0.00
2,1.0,357570,2000.0,0.0,Free,gross,50.00,40.00,1.4,78,2.0,1.0,4806.5,77.0,28.843957,30.181532,7.749170,2.59
3,1.0,168275,1512.0,0.0,Free,actual,54.00,28.00,1.5,72,0.0,0.0,2328.0,71.0,27.250909,23.849315,7.788767,4.02
4,1.0,188000,1225.0,0.0,Free,net,35.00,35.00,1.2,75,3.0,2.0,1971.1,74.0,27.715336,14.010556,7.923788,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2.0,300000,4999.9,0.0,Free,actual,70.71,70.71,1.5,88,0.0,0.0,7003.0,89.0,28.609138,36.200000,7.668314,0.00
2496,1.0,100000,900.0,0.0,Free,net,45.00,20.00,1.5,100,14.0,7.0,1340.5,87.0,29.144635,24.300769,7.317885,0.33
2497,1.0,30716,86.0,0.0,Free,gross,10.71,8.03,1.0,89,3.0,1.0,470.0,78.0,28.704391,5.713902,7.940149,1.54
2498,1.0,150000,1600.0,0.0,Free,gross,45.00,35.00,2.0,54,2.0,1.0,663.0,54.0,28.628733,16.994333,7.657651,0.00


In [73]:
#traint test definition
X = df.drop(['cycle_id','weight','size','avg_weight_har','selling_price','total_biocunt','last_sampled_weight','generated_date','survival_rate'], axis=1)
y_1 = df['survival_rate']
y_2 = df['avg_weight_har']
y_3 = df['selling_price']
y_4 = df['weight']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_1, test_size=0.2, random_state=42)

# Define preprocessing for numerical and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

#
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))  # Change to AdaBoostRegressor() if needed
])

# Define hyperparameter grid for tuning
param_grid = {
    'regressor': [RandomForestRegressor(random_state=42)],
    'regressor__n_estimators': [200,400],
    'regressor__max_depth': [5,10, 20] if isinstance(pipeline.named_steps['regressor'], RandomForestRegressor) else [None]
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train, y_train)

# Evaluate the model
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Best model
print(f'Best model: {grid_search.best_estimator_}')
# model path
version = "version_{}".format(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
pickle_file_path = 'model_repo\sr\model_sr_{}.pkl'.format(version)

# Export the best_model to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(grid_search, file)

Mean Squared Error: 1005.2688153258376
Best model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['total_seed', 'area', 'initial_age', 'pond_length', 'pond_width',
       'pond_depth', 'cyc_dur', 'fasting', 'cons_fast_count', 'quantity',
       'cons_feed_count', 'morning_temperature', 'morning_salinity',
       'morning_pH', 'avg_growth_rate'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                

In [74]:
best_model = grid_search.best_estimator_
feature_names = numeric_features.tolist() + best_model.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features).tolist()
feature_importances = best_model.named_steps['regressor'].feature_importances_
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

In [75]:
feature_importances_df.to_excel(r'Report\feature_importances_sr_model_randomforest.xlsx', index=False)

# Target 2: average weight harverst in grams

In [77]:
#traint test definition
X = df.drop(['cycle_id','weight','size','avg_weight_har','selling_price','total_biocunt','last_sampled_weight','generated_date','survival_rate'], axis=1)
y_1 = df['survival_rate']
y_2 = df['avg_weight_har']
y_3 = df['selling_price']
y_4 = df['weight']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_2, test_size=0.2, random_state=42)

# Define preprocessing for numerical and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

#
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))  # Change to AdaBoostRegressor() if needed
])

# Define hyperparameter grid for tuning
param_grid = {
    'regressor': [RandomForestRegressor(random_state=42)],
    'regressor__n_estimators': [200,400,500],
    'regressor__max_depth': [5,10, 20] if isinstance(pipeline.named_steps['regressor'], RandomForestRegressor) else [None]
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train, y_train)

# Evaluate the model
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Best model
print(f'Best model: {grid_search.best_estimator_}')

# model path
version = "version_{}".format(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
pickle_file_path = 'model_repo\\avg_harvest\model_avghar_{}.pkl'.format(version)

# Export the best_model to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(grid_search, file)

Mean Squared Error: 18.98047897226406
Best model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['total_seed', 'area', 'initial_age', 'pond_length', 'pond_width',
       'pond_depth', 'cyc_dur', 'fasting', 'cons_fast_count', 'quantity',
       'cons_feed_count', 'morning_temperature', 'morning_salinity',
       'morning_pH', 'avg_growth_rate'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                 

In [78]:
best_model = grid_search.best_estimator_
feature_names = numeric_features.tolist() + best_model.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features).tolist()
feature_importances = best_model.named_steps['regressor'].feature_importances_
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

In [79]:
feature_importances_df.to_excel(r'Report\feature_importances_avghar_model_randomforest.xlsx', index=False)

# Target3: Selling Price

In [80]:
#traint test definition
X = df.drop(['cycle_id','weight','size','avg_weight_har','selling_price','total_biocunt','last_sampled_weight','generated_date','survival_rate'], axis=1)
y_1 = df['survival_rate']
y_2 = df['avg_weight_har']
y_3 = df['selling_price']
y_4 = df['weight']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_3, test_size=0.2, random_state=42)

# Define preprocessing for numerical and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

#
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))  # Change to AdaBoostRegressor() if needed
])

# Define hyperparameter grid for tuning
param_grid = {
    'regressor': [RandomForestRegressor(random_state=42)],
    'regressor__n_estimators': [200,400,500],
    'regressor__max_depth': [5,10, 20] if isinstance(pipeline.named_steps['regressor'], RandomForestRegressor) else [None]
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train, y_train)

# Evaluate the model
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Best model
print(f'Best model: {grid_search.best_estimator_}')

version = "version_{}".format(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
pickle_file_path = 'model_repo\\sellprice\model_sellprice_{}.pkl'.format(version)

# Export the best_model to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(grid_search, file)

Mean Squared Error: 5.842517761906155e+16
Best model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['total_seed', 'area', 'initial_age', 'pond_length', 'pond_width',
       'pond_depth', 'cyc_dur', 'fasting', 'cons_fast_count', 'quantity',
       'cons_feed_count', 'morning_temperature', 'morning_salinity',
       'morning_pH', 'avg_growth_rate'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                             

In [81]:
best_model = grid_search.best_estimator_
feature_names = numeric_features.tolist() + best_model.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features).tolist()
feature_importances = best_model.named_steps['regressor'].feature_importances_
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

In [82]:
feature_importances_df.to_excel(r'Report\feature_importances_sellprice_model_randomforest.xlsx', index=False)

# Target 4: Biomass

In [83]:
#traint test definition
X = df.drop(['cycle_id','weight','size','avg_weight_har','selling_price','total_biocunt','last_sampled_weight','generated_date','survival_rate'], axis=1)
y_1 = df['survival_rate']
y_2 = df['avg_weight_har']
y_3 = df['selling_price']
y_4 = df['weight']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_4, test_size=0.2, random_state=42)

# Define preprocessing for numerical and categorical features
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

#
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a pipeline with preprocessing and model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))  # Change to AdaBoostRegressor() if needed
])

# Define hyperparameter grid for tuning
param_grid = {
    'regressor': [RandomForestRegressor(random_state=42)],
    'regressor__n_estimators': [200,400,500],
    'regressor__max_depth': [5,10, 20] if isinstance(pipeline.named_steps['regressor'], RandomForestRegressor) else [None]
}

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')

# Fit the model
grid_search.fit(X_train, y_train)

# Evaluate the model
y_pred = grid_search.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Best model
print(f'Best model: {grid_search.best_estimator_}')

version = "version_{}".format(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
pickle_file_path = 'model_repo\\biomass\model_biomass_{}.pkl'.format(version)

# Export the best_model to a pickle file
with open(pickle_file_path, 'wb') as file:
    pickle.dump(grid_search, file)

Mean Squared Error: 73530018.1098156
Best model: Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['total_seed', 'area', 'initial_age', 'pond_length', 'pond_width',
       'pond_depth', 'cyc_dur', 'fasting', 'cons_fast_count', 'quantity',
       'cons_feed_count', 'morning_temperature', 'morning_salinity',
       'morning_pH', 'avg_growth_rate'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                  

In [84]:
best_model = grid_search.best_estimator_
feature_names = numeric_features.tolist() + best_model.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features).tolist()
feature_importances = best_model.named_steps['regressor'].feature_importances_
feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

feature_importances_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

In [85]:
feature_importances_df.to_excel(r'Report\feature_importances_biomass_model_randomforest.xlsx', index=False)